In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import json
import requests

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

global_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json').text)
local_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-municipalities-2018-geo.json').text)
#--#
url = 'https://raw.githubusercontent.com/guebin/DV2022/main/posts/Energy/{}.csv'
prov = ['Seoul', 'Busan', 'Daegu', 'Incheon', 
        'Gwangju', 'Daejeon', 'Ulsan', 'Sejongsi', 
        'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 
        'Chungcheongnam-do', 'Jeollabuk-do', 'Jeollanam-do', 
        'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do']
df = pd.concat([pd.read_csv(url.format(p+y)).assign(년도=y, 시도=p) for p in prov for y in ['2018', '2019', '2020', '2021']]).reset_index(drop=True)\
.assign(년도 = lambda df: df.년도.astype(int))\
.set_index(['년도','시도','지역']).applymap(lambda x: int(str(x).replace(',','')))\
.reset_index()
df_global = pd.DataFrame([l['properties'] for l in global_dict['features']])
df_local = pd.DataFrame([l['properties'] for l in local_dict['features']])
df_json = df_local.set_index('name')\
.rename(
    {name:name.split('시')[-1] for name in df_local['name'] if ('시' in name) and ('구' in name) and (len(name)>3)}
).reset_index()\
.drop(['base_year','name_eng'],axis=1)\
.rename({'name':'name_local','code':'code_local'},axis=1)\
.assign(code = lambda df: df['code_local'].str[:2])\
.merge(df_global.drop(['base_year','name_eng'],axis=1))
df_left = df.assign(
    시도 = lambda df: df.시도.map({l['properties']['name_eng']:l['properties']['name'] for l in global_dict['features']})
).assign(on = lambda df: df.시도 + '-' + df.지역)
df_right = df_json.assign(on = lambda df: df.name + '-' + df.name_local)\
.drop(['name_local','name'],axis=1)
df2 = df_left.merge(
    df_right.set_index('on')\
    .rename({'인천광역시-남구':'인천광역시-미추홀구'})\
    .reset_index()
).drop(['on'],axis=1)

# 데이터탐색

In [ ]:
import itables 
itables.show(df2,showIndex=False)

# 시각화 {orientation="columns" scrolling="true"}

## Column {width=50%}

In [ ]:
fig = px.choropleth_mapbox(
    geojson = local_dict,
    featureidkey = 'properties.code',
    data_frame = df2.query('년도 == 2018'),
    locations = 'code_local',
    color = '에너지사용량(TOE)/전기',
    hover_data = ['시도','지역'],
    #---#
    mapbox_style="carto-positron",
    range_color = [0,200000],
    center={"lat": 36, "lon": 127.5}, 
    zoom=6,
)
fig.show(config={'scrollZoom':False})

## Column {width=50%}

In [ ]:
seoul_dict = local_dict.copy() 
seoul_dict['features'] = [l for l in seoul_dict['features'] if l['properties']['code'][:2] == '11']
fig = px.choropleth_mapbox(
    geojson = seoul_dict,
    featureidkey = 'properties.code',
    data_frame = df2,
    locations = 'code_local',
    color = '에너지사용량(TOE)/전기',
    animation_frame= '년도',
    hover_data = ['시도','지역'],
    #---#
    mapbox_style="carto-positron",
    range_color = [0,400000],
    center={"lat": 37.5642135, "lon": 127.0016985},
    zoom=9,
)
fig.show(config={'scrollZoom':False})

In [ ]:
fig = df2.drop(['code_local','code'],axis=1)\
.set_index(['년도','시도','지역','건물동수','연면적']).stack().reset_index()\
.rename({'level_5':'에너지종류', 0:'에너지사용량'},axis=1)\
.assign(에너지종류 = lambda df: df['에너지종류'].str.split('/').str[-1])\
.groupby(['년도','시도','에너지종류']).agg({'에너지사용량':'sum'})\
.stack().reset_index()\
.rename({0:'에너지사용량'},axis=1)\
.plot.area(
    x='년도',y='에너지사용량',
    color='시도',
    facet_col='에너지종류'
)
fig.update_layout(
    xaxis_domain=[0.0, 0.25],
    xaxis2_domain=[0.35, 0.60],
    xaxis3_domain=[0.70, 0.95]
)

# GDP

In [ ]:
df = px.data.gapminder()

## Row {height=60%} # <2>

```{{python}}  # <3>
#| title: GDP and Life Expectancy 
px.scatter(  
  df, x="gdpPercap", y="lifeExp", 
  animation_frame="year", animation_group="country", 
  size="pop", color="continent", hover_name="country",
  facet_col="continent", log_x=True, size_max=45, 
  range_x=[100,100000], range_y=[25,90] 
)  
``` # <3>

## Row {height=40%}

```{{python}} # <4>
#| title: Population
px.area(
  df, x="year", y="pop", 
  color="continent", line_group="country"
)
```

```{{python}}
#| title: Life Expectancy
px.line(
  df, x="year", y="lifeExp", 
  color="continent", line_group="country"
)
``` # <4>
````

1.  The document options define the `title` and `author` for the navigation bar as well as specifying the use of the `dashboard` format.
2.  Rows and columns are defined using headings. In this example we define two rows and specify their relative sizes using the `height` option.
3.  Computational cells become cards that live within rows or columns. Cards can have an optional title (which here we specify using the `title` option).
4.  The second row includes two computational cells, which are automatically split into two side by side cards.

## Getting Started

### Step 1: Install Quarto Pre-Release

Dashboards are a feature in the upcoming 1.4 release of Quarto and are still under active development. Before you get started, make sure you install the **latest pre-release** version of Quarto.

If you are using Quarto within a Python environment you can install the pre-release with `pip` as follows:

```{.bash filename="Terminal"}
pip install git+https://github.com/quarto-dev/quarto-cli
```

Alternatively, run the global installer for your platform:

{{< include ../download/_download-pre.md >}}

```{=html}
<style type="text/css">
#download-pre-table {
  display: none;
}
</style>
```

You can find release notes and installers for all platforms at <https://quarto.org/docs/download/prerelease.html>

### Step 2: Learn the Basics

Start by learning how to lay out your dashboard and populate it with content:

[Dashboard Layout](layout.qmd) shows you how to control the navigation bar, and how to arrange your content across pages, rows, columns, tabsets, and cards.

[Data Display](data-display.qmd) shows you how to display data in your dashboard as plots, tables, value boxes, and text.

### Step 3: Explore Further

Once you've mastered the basics, check out these additional articles to learn more.

[Examples](examples/index.qmd) provides a gallery of example dashboards you can use as inspiration for your own.

[Inputs](inputs.qmd) demonstrates various ways to layout inputs for interactive dashboards (sidebars, toolbars, attaching inputs directly to cards, etc.)

[Theming](theming.qmd) describes the various way to customize the fonts, colors, layout and other aspects of dashboard appearance.

[Parameters](parameters.qmd) explains how to create dashboard variants by defining parameters and providing distinct values for them on the command line.

[Deployment](deployment.qmd) covers how to deploy both static dashboards (which require only a web host, but not a server) and Shiny dashboards (which require a Shiny Server).

[Interactivity](interactivity/index.qmd) explores the various ways to create interactive dashboards that enable more flexible data exploration.
